<a href="https://colab.research.google.com/github/Bullbotbam/autmhq_job_board/blob/main/automated_job_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 5.5 MB/s 
     |████████████████████████████████| 138 kB 45.0 MB/s 
     |████████████████████████████████| 358 kB 14.7 MB/s 
     |████████████████████████████████| 55 kB 1.3 MB/s 
     |████████████████████████████████| 4.0 MB 34.6 MB/s 
     |████████████████████████████████| 58 kB 4.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException,TimeoutException,StaleElementReferenceException

import time
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
import re 
from datetime import date
from datetime import datetime
from datetime import timedelta
import sys

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
def make_noise():
    sys.stdout.write('\a')
    sys.stdout.flush()

## Helper functions

### Formatting the date:
def split_date(x):
    m = re.search(r'\d+[+]? \w+ \w+' ,str(x))
    if m:
        return m.group()
    else:
        return " "
    
def convert_date(x):
    m = re.search(r'\d+' ,str(x))
    if m:
        dt = datetime.today()-timedelta(int(m.group()))
        dt = dt.strftime('%m/%d/%Y')  #time_dict)
        return dt
    else:
        return " "
    
### Finding Zip and City
def find_zip(x):
    z = re.search(r'\d+',x )
    if z:
         return(z.group())
    else :
        return " "

## Indeed 

In [ ]:
def scrape_indeed_jobs(keyword, location, n_pages):

    url = f"https://www.indeed.com/?from=gnav-homepage"
    ## Create an instance of the chrome webdriver. This is to interact with the website and test my code
    s = Service ('C:\Program Files (x86)\chromedriver_win32\chromedriver.exe')
    wd = webdriver.Chrome(service = s)
    wait = WebDriverWait(wd, 5)

    ## Open Indeed Webpage 
    wd.get(url)

    ## Enter search parameters
    what_search = wd.find_element(By.ID, 'text-input-what')
    what_search.send_keys(keyword)

    where_search = wd.find_element(By.ID, 'text-input-where')
    where_search.send_keys(Keys.CONTROL + "a")
    where_search.send_keys(Keys.DELETE)
    where_search.send_keys(location)
    where_search.send_keys(Keys.RETURN)

    base_url = wd.current_url

    ## Initialise the different lists:
    job_id =  []
    job_title = []
    seniority = []
    emp_type=[]
    job_link = []
    industries = []
    company_names = []
    company_names2 = []
    job_date =[]
    job_location = []
    job_salary = []
    job_qual = []
    job_description = []
    
    def extract(wait,job_id,job_title,seniority,emp_type,job_link,industries,company_names,company_names2,job_date,job_location,job_salary,job_qual,job_description):
        results = wait.until(
                EC.presence_of_element_located((By.ID, "mosaic-provider-jobcards")))
        jobs = results.find_elements(By.TAG_NAME,"a")
        for job in jobs:
            try:
                href = job.get_attribute("href")
                job_link.append(href)
            except(NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                job_link.append('')

            ## Find the job's id
            try :
                ids = job.get_attribute("id")
                job_id.append(ids)
            except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                job_id.append('')

            ## Find date posted
            try:
                date = job.find_element(By.CLASS_NAME,'date').text
                # print('date', date)
                job_date.append(date)
            except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                job_date.append("")

               ## Find the company name alternative :
            try:
                company2 = job.find_element(By.XPATH,'//a[@data-tn-element = "companyName"]').get_attribute("textContent")
                company_names2.append(company2)                
            except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                company_names2.append('')

            try:

                job.find_element(By.CSS_SELECTOR, 'div').click()
                time.sleep(2)
                iframe = results.find_element(By.ID,"vjs-container").find_element(By.CSS_SELECTOR,'iframe')
                wd.switch_to.frame(iframe)

                        ## Find the job's title 
                try:
                    title = wait.until(EC.presence_of_element_located((By.TAG_NAME,"h1"))).get_attribute("textContent")
                    title = title.replace(" - job post", "")
                    job_title.append(title)
                        #print(title)   
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                    job_title.append('')


                    ## Find the company name:
                try:
                    company = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewJobSSRRoot"]/div[1]/div[1]/div/div/div/div[1]/div/div[1]/div[1]/div[1]/div[2]/div/div/div/div[1]/div[2]/div/a'))).get_attribute('innerText')
                        #print(f'company,{company}')//*[@id="viewJobSSRRoot"]/div[1]/div[1]/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[2]/div/div/div/div[1]/div[2]/div/a
                    company_names.append(company)                
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                    company_names.append('') 

                    ## Find the location 
                try:
                    loc = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="viewJobSSRRoot"]/div[1]/div[1]/div/div/div/div[1]/div/div[1]/div[1]/div[1]/div[2]/div/div/div/div[2]/div'))).get_attribute('innerText')
                       # print(f'location, {loc}')                                  //*[@id="viewJobSSRRoot"]/div[1]/div[1]/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[2]/div/div/div/div[2]/div
                    job_location.append(loc)
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                    job_location.append('')

                    ## Find the salary:
                try:
                    salary  = wait.until(
                        EC.presence_of_element_located((By.XPATH,'//*[@id="jobDetailsSection"]/div[2]/span'))).get_attribute('innerText')
                    job_salary.append(salary)
                       # print(f'salary,{salary}')
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                    job_salary.append('')

                    ## Find Employment type: 
                try:
                    emp = wait.until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="jobDetailsSection"]/div[3]/div[2]'))).get_attribute('innerText')             
                    emp_type.append(emp)
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                    emp_type.append("")

                     ## Qualifications            
                try:
                    qual = wait.until(
                        EC.presence_of_element_located((By.CLASS_NAME, "jobsearch-ReqAndQualSection-item--wrapper"))).get_attribute('innerText')
                       # print('qual',qual)
                    job_qual.append(qual)
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException):
                    job_qual.append('')

                    ## Job summary:
                try:
                    summary = wait.until(
                        EC.presence_of_element_located((By.XPATH,'//*[@id="jobDescriptionText"]'))).get_attribute('innerText')
                       #print(f'summary,{summary}')
                    job_description.append(summary)
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException):
                    job_description.append('')


                    #print("--------------")
                wd.switch_to.default_content()

            except:
                job_title.append('')
                seniority.append('')
                emp_type.append('')
                industries.append('')
                company_names.append('')
                job_location.append('')
                job_salary.append('')
                job_qual.append('')
                job_description.append('')   

        return job_id,job_title,seniority,emp_type,job_link,industries,company_names,company_names2,job_date,job_location,job_salary,job_qual,job_description
    
    ## Iterate over the different pages 
    i=1
    count = 1

    try:
        wait.until(EC.element_to_be_clickable((By.ID,'popover-x'))).click()       
    except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
        print("no pop up")

    job_id,job_title,seniority,emp_type,job_link,industries,company_names,company_names2,job_date,job_location,job_salary,job_qual,job_description = extract(wait,job_id,job_title,seniority,emp_type,job_link,industries,company_names,company_names2,job_date,job_location,job_salary,job_qual,job_description)
    while i<= n_pages:
        wd.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        try:
            #time.sleep(1)
            button = wd.find_element(By.XPATH,f'//*[@id="resultsCol"]/nav/div/ul/li[{i}]/a').click()
            try:
                wait.until(EC.element_to_be_clickable((By.ID,'popover-x'))).click()       
            except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                print("no pop up")
            job_id,job_title,seniority,emp_type,job_link,industries,company_names,job_date,job_location,job_salary,job_qual,job_description = extract(wait,job_id,job_title,seniority,emp_type,job_link,industries,company_names,company_names2,job_date,job_location,job_salary,job_qual,job_description)
            time.sleep(1)
            #wd.back()
            count += 1
            print(f'COUNT = {count}')
        except:
            print("End of pages scaped")
        i+=1

    ## Cleaning the dataframe 
    job_list = {'ID' : job_id,
                'Title': job_title,
                'Date': job_date, 
                'Company': company_names,
                'Company2': company_names2,
                'Location': job_location,
                'Salary': job_salary,
                #'Seniority': seniority,
                'Employment Type': emp_type,
                #'Function': job_func,
                #'Industry': industries,
                'Qualification': job_qual,
                'Job description':job_description,
                'Link': job_link}

    df = pd.DataFrame(job_list)
    df = df.replace('',np.nan)
    df= df.dropna(subset=['Title'])
    df['City'] = df['Location'].apply(lambda x: str(x).split(',')[0])
    df['Zip Code'] = df['Location'].apply(lambda x: find_zip(str(x)))
    df['Date Posted'] = df['Date'].apply(lambda x: convert_date(x))    
    df['Date'] = df['Date'].apply(lambda x: split_date(x))

    #df.to_csv("WebScrapper_Indeed"+ str(time.time())+'.csv')

    ## Homogenize the data frame:
    df['Seniority'] = np.nan
    df['Function'] = np.nan
    df['Industry'] = np.nan
    df['Source'] = 'Indeed'

    cols = [ 'ID', 'Title', 'Date', 'Date Posted', 'Company', 'Company2','Location',
                'Zip Code' ,'Salary','Employment Type', 'Qualification', 'Seniority', 'Function', 'Industry','Job description', 'Link', 'Source' ]
    df= df[cols]
    return df

## Monster

In [ ]:
def scrape_monster_jobs(keyword, location, n_pages):
    url = f"https://www.monster.com/jobs"

    ## Create an instance of the webdriver 
    s = Service('C:\Program Files (x86)\chromedriver_win32\chromedriver.exe')
    wd = webdriver.Chrome(service = s)

    ## Open the Monster Webpage 
    wd.get(url)
    wait = WebDriverWait(wd,10)

    ## Input the Job title and location
    what_search = wd.find_element(By.ID, "search-job" )
    what_search.send_keys(keyword)

    where_search = wd.find_element(By.ID, "search-location")
    where_search.send_keys(location)
    where_search.send_keys(Keys.RETURN)

    time.sleep(5)

    ## Execute script to scroll to bottom of the page 
    i=1
    while (i <= n_pages):
        wd.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        #WebDriverWait(wd, 5).until(EC.element_to_be_clickable((By.XPATH,'/*[@id="__next"]/div[3]/div[1]/main/div[2]/nav/section[1]/div[2]/div/div/div/div[64]/button'))).click()  
        i+=1

    wd.switch_to.window(wd.window_handles[0])
    jobs = wd.find_element(By.XPATH,'//*[@id="__next"]/div[3]/main/div[2]/nav/section[1]/div[2]/div/div/div').find_elements(By.CSS_SELECTOR,'div')
    n_jobs = len(jobs)
    print(n_jobs)

    ## Initialise the different lists:
    job_id =  []
    job_title = []
    seniority = []
    emp_type=[]
    job_link = []
    industries = []
    company_names = []
    job_date =[]
    job_location = []
    job_salary = []
    job_qual = []
    job_description = []
    
    def scrape_jobs(n_jobs,job_id,job_title,seniority,emp_type,job_link,industries,company_names,job_date,job_location,job_salary,job_qual,job_description):
        for i in range(1,n_jobs):         
            try:
                wd.execute_script("window.scrollTo(0,document.body.scrollHeight);")
                job = wd.find_element(By.XPATH,f' //*[@id="__next"]/div[3]/main/div[2]/nav/section[1]/div[2]/div/div/div/div[{i}]/a/div/div[1]').click()      
                wd.switch_to.window(wd.window_handles[1])
                try:
                    title = wd.find_element(By.TAG_NAME, "h1").text
                    job_title.append(title)
                    print(title)
                except:
                    job_title.append('')
                try:
                    date = wd.find_element(By.XPATH, "//div[@data-test-id='svx-jobview-posted']").text
                    print(date)
                            #date = wd.find_element(By.XPATH,'//*[@id="details-table"]').find_element(By.CSS_SELECTOR, 'div.detailsstyles__DetailsTableDetailBody-sc-1deoovj-5.hDKBZO').text
                    job_date.append(date)
                except:
                    job_date.append('')
                try:
                    company = wd.find_element(By.TAG_NAME, "h2").text
                    company_names.append(company)
                except:
                    company_names.append('')
                        #print(company)
                try:
                    location = wd.find_element(By.TAG_NAME, "h3").text
                    job_location.append(location)
                except:
                    job_location.append('')
                        #print(location)
                try:
                    link = wd.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
                    job_link.append(link)
                except:
                        job_link.append('')
                        #print(link)
                try:
                    salary= wd.find_element(By.XPATH,'//*[@id="details-table"]').find_element(By.CSS_SELECTOR,'div.salarystyle__SalaryContainer-sc-1kub5et-7.TQDXU > div').text
                            #salary = wd.find_element(By.CLASS_NAME, "salarystyle__SalaryBody-sc-1kub5et-8 jeLUTC").text
                    job_salary.append(salary)
                except:
                    job_salary.append('')

                try:
                           # emp = wd.find_element(By.XPATH, "//div[@data-test-id='svx-jobview-location']").text
                    emp =wd.find_element(By.XPATH, "//div[@data-test-id='svx-jobview-employmenttype']").text
                    emp_type.append(emp)
                except:
                    emp_type.append('')

                try:
                    description = wd.find_element(By.XPATH,'//*[@id="jobview-container"]/div[1]/div/div[2]/div').text
                    job_description.append(description)
                        #print(description)
                except:
                    job_description.append('')

                wd.close() 
                wd.switch_to.window(wd.window_handles[0]) 
                time.sleep(5)

            except:
                pass         #print(i)        
        return  job_id,job_title,seniority,emp_type,job_link,industries,company_names,job_date,job_location,job_salary,job_qual,job_description

    job_id,job_title,seniority,emp_type,job_link,industries,company_names,job_date,job_location,job_salary,job_qual,job_description=scrape_jobs(n_jobs,job_id,job_title,seniority,emp_type,job_link,industries,company_names,job_date,job_location,job_salary,job_qual,job_description)

    ## Clean out the dataframe 
    job_list = {#'ID' : job_id,
                'Title': job_title,
                'Date': job_date, 
                'Company': company_names,
                'Location': job_location,
                'Salary': job_salary,
                #'Seniority': seniority,
                'Employment Type': emp_type,
                #'Function': job_func,
                #'Industry': industries,
                #'Qualification': job_qual,
                'Job description':job_description,
                'Link': job_link}
    df = pd.DataFrame(job_list)

    ## Remove empty lines 
    df = df.replace('',np.nan)
    df= df.dropna(subset=['Title'])

    # Convert the date
    df['Date Posted'] = df['Date'].apply(lambda x: convert_date(x))
    df['Date'] = df['Date'].apply(lambda x: split_date(x))

    ## Add variables that don't exsist in the website 
    df['ID'] = np.nan
    df['Function'] = np.nan
    df['Industry']= np.nan
    df['Qualification'] = np.nan
    df['Seniority'] = np.nan
    df['Industry'] = np.nan
    df['Zip Code'] = np.nan
    df['Company2'] = np.nan
    df['Source'] = 'Monster'
    
    cols = [ 'ID', 'Title', 'Date', 'Date Posted', 'Company', 'Company2','Location',
            'Zip Code' ,'Salary','Employment Type', 'Qualification', 'Seniority', 'Function', 'Industry','Job description', 'Link','Source' ]
    df= df[cols]
    
    return df

## Linkedin

In [ ]:
def scrape_linkedin_jobs(keyword, location, n_pages):

    s = Service ('C:\Program Files (x86)\chromedriver_win32\chromedriver.exe')
    wd = webdriver.Chrome(service = s)
    wait = WebDriverWait(wd, 1)
    actions = ActionChains(wd)
    ## change User Agent at each request 

    username = 'contact@autmhq.org'
    password = "2023GlobalTech"
    url = "https://www.linkedin.com/"
    wd.get(url)


    ## Sign_in in LinkedIn 
    signin = wd.find_element(By.XPATH, '/html/body/nav/div/a[2]').click()
    ids = wd.find_element(By.ID, "username")
    ids.send_keys (username)
    code =  wd.find_element(By.ID, "password") 
    code.send_keys(password)
    code.send_keys(Keys.RETURN)


    wd.find_element(By.XPATH,'//*[@id="ember20"]').click()
    time.sleep(3)
    what_search =  wd.find_element(By.CSS_SELECTOR,("input[id*='jobs-search-box-keyword-id']"))
    what_search.send_keys(keyword)
    time.sleep(2)
    where_search =  wd.find_element(By.CSS_SELECTOR,("input[id*='jobs-search-box-location-id']"))
    time.sleep(2)
    where_search.send_keys(place)
    what_search.send_keys(Keys.RETURN)

    ## Iterate over the number of pages:
    page = 1 
        #while i <= int(no_of_jobs/len(jobs_per_page))+1:
    while page <= n_pages:
        wd.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        page = page+1
        try: 
            wd.find_element(By.XPATH,'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/section/div/ul/li[{page}]').click()
            time.sleep(5)
        except:
            pass

        time.sleep(5)

    wd.maximize_window()
     ## Find the number of jobs on one page:
    job_list = wd.find_element(By.CLASS_NAME,'jobs-search-results__list')
    jobs = job_list.find_elements(By.CLASS_NAME,'jobs-search-results__list-item')
    n_jobs = len(jobs)
    #print(n_jobs)
    init_url= wd.current_url
    #print(init_url)

    ## Initialize the different lists
    job_id = []
    job_title = []
    company = []
    job_location = []
    job_date = []
    job_link = []
    job_description = []
    seniority = []
    emp_type=[]
    job_func = []
    industries = []
    company_names = []
    
    i= 1
    for job in jobs:
        try:
            ids = job.get_attribute("data-job-id")
            job_id.append(ids)
        except:
            job_id.append('')
            print('id fail')
        try:
            title = wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{i}]/div/div/div[1]/div[2]/div[1]/a'))).text
            job_title.append(title)
        except:
            job_title.append('')
            print('title fail')
        try:
            comp= wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{i}]/div/div/div[1]/div[2]/div[2]/a'))).text
            company.append(comp)

        except:
            company.append('')
            print('company fail')
        try:
            location = wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{i}]/div/div/div[1]/div[2]/div[3]/ul/li'))).text  
            job_location.append(location)

        except:
            job_location.append('')
            print('location fail')

        try:
            uid = job.get_attribute("id")
            path = r'//*[@id = "{}"]/div/div'.format(uid)
            #print(uid,path)
            wd.find_element(By.XPATH,path).click()
            cur_url = wd.current_url

            try: 
                link = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[1]/div/div[2]/a'))).get_attribute('href')
                print(link)
                job_link.append(link)
            except:
                job_link.append('')

            try:
                emp = wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[1]/div/div[2]/div[2]/ul/li[1]/span'))).text
                print(emp)
                emp_type.append(emp)
               # print("Emp type  ",emp_type)
            except:
                emp_type.apppend('')

            try:
                industry = wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[1]/div/div[2]/div[2]/ul/li[2]/span/a'))).text
                industries.append(industry)
              #  print("Industry   ",industry)
            except:
                industries.append('')

            try:
                age = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[1]/div/div[2]/div[1]/span[2]/span[1]'))).text
                job_date.append(age)
            except:
                job_date.append('')

            try:
                description = wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[2]/article/div/div[1]/span'))).text
                job_description.append(description)
            except:
                job_description.append('')

            #if init_url != cur_url:
             #   wd.back()


        except:
            print('fail')
            job_link.append('')
            job_date.append('')
            job_description.append('')
            emp_type.append('')
            industries.append('')

       # if init_url != cur_url:
        #    wd.back()



        i= i+1
        print(i)


    df = pd.DataFrame({ 'ID' : job_id,
                             'Title': job_title,
                             'Date Posted': job_date, 
                             'Company': company,
                             'Location': job_location,
                             'Employment Type': emp_type,
                             'Industry': industries,
                             'Link': job_link,
                          #   'Job description': job_description
                        })

    df['Company2'] = np.nan
    df['Zip Code'] = np.nan
    df['Date'] = np.nan
    df['Qualification'] = np.nan
    df['Salary'] = np.nan
    df['Source'] = 'LinkedIn'

    cols = [ 'ID', 'Title', 'Date', 'Date Posted', 'Company', 'Company2','Location',
                'Zip Code' ,'Salary','Employment Type', 'Qualification', 'Industry','Job description', 'Link','Source']
    #df= df[cols]

    return df

## TODO 
For each data frame scrapped:
 <ul>
  <li> Write a script that scrapes jobs every day </li>
  <li> Check that there is no duplicate data  </li>
  <li> Copy the cleaned data and export to our master list  </li>
  <li> Check that there's no duplicate data when we export the new jobs  </li>
  <li> Remove jobs that are too old </li>
  <li> Scrape jobs: just get the first page for each job</li>
 </ul>


In [ ]:
## Input the keywords 
#keyword = input("Enter the postions you are scraping")
#place = input ("Enter the job's location ")
keyword = "Software Engineer"
place = "Austin TEXAS"

In [ ]:
def update_master_df():
    master_df = pd.read_csv("Master_job_list.csv")
    
    # Remove duplicates
    master_df.drop_duplicates(inplace = True)
    
    # Remove jobs that are too old (posted more than 2 weeks ago)
   # date = 
    master_df = master_df[(master_df['Date Posted'] > (date.today() -timedelta(14)).strftime('%m/%d/%Y')) & (master_df['Date Posted'] <=  (date.today()+timedelta(14)).strftime('%m/%d/%Y'))]
    
    # Save master_df
    master_df.to_csv("Master_job_list.csv",index = False)
    

In [ ]:
## run scripts to scrape data from LinkedIn, Monster, Indeed 
while(True):
    linkedin_df = scrape_linkedin_jobs(keyword, place ,1)
    monster_df= scrape_monster_jobs(keyword, place , 1)
    indeed_df= scrape_indeed_jobs(keyword, place , 1)
    make_noise()
    
    ## Merge scrapped data in one file 
    df = pd.concat([linkedin_df,monster_df,indeed_df])
  
    ## Remove the duplicates from the total dataframe 
    df.drop_duplicates(inplace = True)

    ## Export cleaned data frame to our master list 
    #df.to_csv("Master_job_list.csv", index = False, header = True) ## To be used for first run 
    df.to_csv("Master_job_list.csv", mode = 'a', index = False, header = False) 

    ## Remove jobs that are too old: 
    update_master_df()
    #time.sleep(86400)

WebDriverException: ignored

In [ ]:
    ## Merge scrapped data in one file 
df = pd.concat([linkedin_df,monster_df,indeed_df])
   # df = pd.concat([monster_df,indeed_df])
    ## Remove the duplicates from the total dataframe 
df.drop_duplicates(inplace = True)

    ## Export cleaned data frame to our master list #df.to_csv("Master_job_list.csv", index = False, header = True) 
df.to_csv("Master_job_list.csv", mode = 'a', index = False, header = False) 

    ## Remove jobs that are too old: 
    #df
update_master_df()


In [ ]:
## Next Steps 
Extract salary, Education level from the job description